In [0]:
### Get Required Files from Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
project_path='/content/drive/My Drive/Project/'


In [0]:
### Read Data "Sarcasm_Headlines_Dataset.json". Explore the data and get some insights about the data.

In [0]:
import pandas as pd
import json as js

In [0]:
import os
import glob
os.chdir(project_path)
os.listdir()

['glove.6B.zip',
 'glove.6B.zip.1',
 'Sarcasm_Headlines_Dataset.json',
 'glove.6B.50d.txt',
 'glove.6B.100d.txt',
 'glove.6B.200d.txt',
 'glove.6B.300d.txt']

In [0]:
data=pd.read_json('Sarcasm_Headlines_Dataset.json',lines=True)

In [0]:
data.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [0]:
sentences = data["headline"].values
sentences[:5]

array(["former versace store clerk sues over secret 'black code' for minority shoppers",
       "the 'roseanne' revival catches up to our thorny political mood, for better and worse",
       "mom starting to fear son's web series closest thing she will have to grandchild",
       'boehner just wants wife to listen, not come up with alternative debt-reduction ideas',
       'j.k. rowling wishes snape happy birthday in the most magical way'],
      dtype=object)

In [0]:
labels = data["is_sarcastic"].values
labels[:5]

array([0, 0, 1, 1, 0])

In [0]:
urls = data["article_link"].values
urls[:5]

array(['https://www.huffingtonpost.com/entry/versace-black-code_us_5861fbefe4b0de3a08f600d5',
       'https://www.huffingtonpost.com/entry/roseanne-revival-review_us_5ab3a497e4b054d118e04365',
       'https://local.theonion.com/mom-starting-to-fear-son-s-web-series-closest-thing-she-1819576697',
       'https://politics.theonion.com/boehner-just-wants-wife-to-listen-not-come-up-with-alt-1819574302',
       'https://www.huffingtonpost.com/entry/jk-rowling-wishes-snape-happy-birthday_us_569117c4e4b0cad15e64fdcb'],
      dtype=object)

In [0]:
### Drop article_link from dataset.
df=data.drop(['article_link'], axis = 1)
df


,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0
...,...,...
26704,american politics in moral free-fall,0
26705,america's best 20 hikes,0
26706,reparations and obama,0
26707,israeli ban targeting boycott supporters raise...,0


In [0]:
df.shape

(26709, 2)

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26709 entries, 0 to 26708
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   headline      26709 non-null  object
 1   is_sarcastic  26709 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 417.5+ KB


In [0]:
df.describe(include="all")

,headline,is_sarcastic
count,26709,26709.000000
unique,26602,NaN
top,the 20 funniest tweets from women this week,NaN
freq,10,NaN
mean,NaN,0.438953
std,NaN,0.496269
min,NaN,0.000000
25%,NaN,0.000000
50%,NaN,0.000000
75%,NaN,1.000000


In [0]:
### Get the Length of each line and find the maximum length
df['len'] = df['headline'].apply(lambda x: len(x.split(" ")))
# using len() + key argument + max() 
res = len(max(df, key = len)) 
  
# printing result 
print("Length of maximum string is : " + str(res)) 
df.head()
max_length=max([len(headline.split())for headline in df['headline']])
print('Maximum Length:',max_length)

Length of maximum string is : 12
Maximum Length: 39


In [0]:
print("checking null value:\n",df.isna().sum())


checking null value:
 headline        0
is_sarcastic    0
len             0
dtype: int64


In [0]:
words = "./glove.6B.200d.txt"
for item in df.headline:
    for inner_item in item.lower().split():
        words+=inner_item+" "
        
from wordcloud import WordCloud, STOPWORDS 
stopwords = set(STOPWORDS) 
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(words) 
  


In [0]:
### Import required modules required for modelling.

In [0]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential

In [0]:
max_features = 10000
maxlen = 39
embedding_size = 200

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(df['headline']))
X = tokenizer.texts_to_sequences(df['headline'])
X = pad_sequences(X, maxlen = maxlen)
y = df['is_sarcastic']

In [0]:
X = tokenizer.texts_to_sequences(df['headline'])
X = pad_sequences(X, maxlen = maxlen)
y = np.asarray(df['is_sarcastic'])

print("Number of Samples:", len(X))
print(X[0])
print("Number of Labels: ", len(y))
print(y[0])

Number of Samples: 26709
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0  307  678 3336 2297   47  381 2575    5 2576 8433]
Number of Labels:  26709
0


In [0]:
glove_file = project_path + "glove.6B.zip"

In [0]:
#Extract Glove embedding zip file
from zipfile import ZipFile
with ZipFile(glove_file, 'r') as z:
  z.extractall()

In [0]:
EMBEDDING_FILE = './glove.6B.200d.txt'
embeddings = {}
for o in open(EMBEDDING_FILE):
    word = o.split(" ")[0]
    # print(word)
    embd = o.split(" ")[1:]
    embd = np.asarray(embd, dtype='float32')
    # print(embd)
    embeddings[word] = embd

In [0]:
max_features = 10000
maxlen = 39
embedding_size = 200

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(df['headline']))
X = tokenizer.texts_to_sequences(df['headline'])
X = pad_sequences(X, maxlen = maxlen)
y = df['is_sarcastic']

In [0]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))

embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embedding_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


In [0]:
model = Sequential()
model.add(Embedding(max_features, embedding_size, weights = [embedding_matrix]))
model.add(Bidirectional(LSTM(128, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(40, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 100
epochs = 5
history = model.fit(X, y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Epoch 1/5
214/214 [==============================] - 76s 354ms/step - loss: 0.6179 - accuracy: 0.6211 - val_loss: 0.4130 - val_accuracy: 0.8295
Epoch 2/5
214/214 [==============================] - 76s 357ms/step - loss: 0.3680 - accuracy: 0.8588 - val_loss: 0.3212 - val_accuracy: 0.8615
Epoch 3/5
214/214 [==============================] - 76s 356ms/step - loss: 0.2477 - accuracy: 0.9137 - val_loss: 0.3551 - val_accuracy: 0.8577
Epoch 4/5
214/214 [==============================] - 79s 368ms/step - loss: 0.1754 - accuracy: 0.9406 - val_loss: 0.3822 - val_accuracy: 0.8620
Epoch 5/5
214/214 [==============================] - 80s 372ms/step - loss: 0.1223 - accuracy: 0.9614 - val_loss: 0.4976 - val_accuracy: 0.8602
